In [2]:
%matplotlib inline
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.utils import Sequence
from keras.utils import load_img
from keras.optimizers import RMSprop
from keras.optimizers import Adam


2023-01-27 22:00:41.774006: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 22:00:43.907300: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /appl/cudnn/v8.3.2.44-prod-cuda-11.5/lib:/appl/cuda/11.6.0/lib64:/appl/python/3.10.7/lib:/appl/gcc/11.3.0-binutils-2.38/lib64:/appl/gcc/11.3.0-binutils-2.38/lib:/lsf/10.1/linux3.10-glibc2.17-x86_64/lib
2023-01-27 22:00:43.907981: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so

In [3]:
df = pd.read_csv('black_background_500x500.csv')
train_df = df[df['ImagePath'].str.contains("train")]
test_df = df[df['ImagePath'].str.contains("test")]
valid_df = df[df['ImagePath'].str.contains("valid")]

input_size = 500

In [4]:
def load_df_to_list(the_df, i_s):
    the_df = the_df.reset_index()
    x_list = np.zeros((len(the_df), i_s, i_s, 1))
    y_list = np.zeros((len(the_df), 3))
    for i, row in the_df.iterrows():
        data = load_img(path = row['ImagePath'], color_mode = "grayscale")
        data = tf.keras.utils.img_to_array(data, data_format="channels_last", dtype="float32")
        data /= 255
        x_list[i] = np.asarray(data)
        y_list[i] = row[['phi1','PHI', 'phi2']].values
    
    return x_list, y_list

In [5]:
# ImagePath,image_name,phi1,PHI,phi2
train_x_list,train_y_list = load_df_to_list(train_df, input_size)
test_x_list,test_y_list   = load_df_to_list(test_df, input_size)
valid_x_list,valid_y_list = load_df_to_list(valid_df, input_size)

In [6]:
train_x_list.shape

(8750, 500, 500, 1)

In [7]:
type(train_x_list)

numpy.ndarray

In [8]:
# inputs = keras.Input(shape=(input_size, input_size, 1))
# x = layers.Conv2D(filters=4, kernel_size=5, activation="relu")(inputs)
# x = layers.MaxPooling2D(pool_size=4)(x)
# x = layers.Conv2D(filters=4, kernel_size=5, activation="relu")(x)
# x = layers.MaxPooling2D(pool_size=4)(x)
# x = layers.Conv2D(filters=4, kernel_size=5, activation="relu")(x)
# x = tf.keras.layers.Dropout(0.5)(x)
# x = layers.Flatten()(x)
# outputs = layers.Dense(3)(x)

# model = keras.Model(inputs=inputs, outputs=outputs)
# model.summary()

inputs = keras.Input(shape=(input_size, input_size, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=4)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=4)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=4)(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(16, activation="relu")(x)
outputs = layers.Dense(3)(x)

model = keras.Model(inputs=inputs, outputs=outputs)


2023-01-27 22:02:06.369275: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 22:02:07.106934: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 84 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:af:00.0, compute capability: 7.0
2023-01-27 22:02:07.136231: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 84.88M (88997888 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory


In [9]:
import keras.backend as K
  
def abs_loss_function(y_true, y_pred):   
    abs_diff = K.abs(y_true - y_pred)
    ones = tf.ones_like(y_true)
    abs_diff_reversed = K.abs(ones - abs_diff )   
    minimum_from_two = tf.math.minimum(abs_diff, abs_diff_reversed) 
    return tf.math.reduce_mean(minimum_from_two, axis=-1)

In [10]:
model.compile(optimizer = RMSprop(learning_rate=0.001),
              loss = abs_loss_function, 
              metrics = [abs_loss_function])

In [11]:
history = model.fit(x=train_x_list,
                    y=train_y_list,
                    batch_size=32,
                    epochs=5,
                    validation_split=0.1,
                    verbose=1)

2023-01-27 22:02:29.801626: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 7.33GiB (rounded to 7875000064)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-01-27 22:02:29.801707: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-01-27 22:02:29.801739: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 24, Chunks in use: 24. 6.0KiB allocated for chunks. 6.0KiB in use in bin. 1.1KiB client-requested in use in bin.
2023-01-27 22:02:29.801766: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 1, Chunks in use: 1. 512B allocated for chunks. 512B in use in bin. 512B client-requested in use in bin.
2023-01-27 22:02:29.801792: I tensorflow/tsl/framework/bfc_allo

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.